# Invoke Bedrock with HealthScribe Summary

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio* on an **`ml.t3.medium`** instance.

## Setup
Before running the remaining cells of this notebook, we need to confirm you have a version of boto3 that supports Bedrock. Any version newer than **`1.28.58`** should be fine. Run the following cell to check the installed version. 

In [17]:
!pip list | grep boto3

boto3                                1.33.0

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


(Optional) Only run this cell if you have an older version of boto3 that does not support Bedrock.

In [ ]:
!pip install botocore boto3 --upgrade

#### List available models in Bedrock

This cell calls **`list_foundation_models`** API in Bedrock filtering only models provided by **`Anthropic`**.

In [ ]:
import boto3, json

bedrock = boto3.client('bedrock', 'us-west-2')
response = bedrock.list_foundation_models(byProvider='anthropic')
print(json.dumps(response.get('modelSummaries'), indent=2))

---

## 3. Send HealthScribe Summary Plan to Bedrock to Extract CPT Codes

In this step we are going to invoke model **`Claude v2`** from **`Anthropic`** on Amazon Bedrock asking the model to provide procedure codes for all relevant procedures listed in the plan generated by HealthScribe. We are going to start with a simple question and will refine the answers with prompt engineering.

In [ ]:
bedrock_runtime = boto3.client('bedrock-runtime', 'us-west-2')

# retreive plan from previous notebook
%store -r plan
print(plan)

In [ ]:
prompt = "what is the CPT code for an x-ray of their lower back"

body = json.dumps({"prompt": "\n\nHuman:" + prompt + "\n\nAssistant:", "temperature": 0, "max_tokens_to_sample": 400})
modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

In [ ]:
prompt = "for each one of the bullet points defined between triple backslashes, list the most common CPT codes applicable: ```" + plan + "```"

body = json.dumps({"prompt": "\n\nHuman:" + prompt + "\n\nAssistant:", "temperature": 0, "max_tokens_to_sample": 400})
modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))